<a href="https://colab.research.google.com/github/sergiomar73/nlp-google-colab/blob/main/Semantic_Textual_Similarity_for_Transcriptions_Dynamic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import spacy
import torch

In [ ]:
model = SentenceTransformer('paraphrase-distilroberta-base-v2')

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
def calculate_embeddings_with_roberta(text):
  embeddings = model.encode(text, convert_to_tensor=True)
  return embeddings

**Process Categories**

In [12]:
categories = """
Hello=Hello, how are you doing today?;Hi, everybody.;Hi;My name's Johnny
What=most advanced conversation intelligence and AI powered coaching platform;a software platform that helps people reach their potential
How=integrated into video conference platforms;record yourself
Who=everyone is having conversations;if you're an entry-level person
Impact=customer experience;communicating with the customer
Bye=Bye.;And I look forward to talking to you again soon. Thank you very much.
"""

categories.splitlines( )
categories_list = [s.strip() for s in categories.splitlines() if s.strip()]

df_phrases = pd.DataFrame(columns=['order', 'category', 'label', 'example'])

for i, category in enumerate(categories_list):
  main = category.split('=')
  label = main[0]
  for example in main[1].split(';'):
    new_row = { 
      'order': i + 1,
      'category': label,
      'label': label,
      'example': example
    }
    df_phrases = df_phrases.append(new_row, ignore_index=True)

df_phrases.head()

,order,category,label,example
0,1,Hello,Hello,"Hello, how are you doing today?"
1,1,Hello,Hello,"Hi, everybody."
2,1,Hello,Hello,Hi
3,1,Hello,Hello,My name's Johnny
4,2,What,What,most advanced conversation intelligence and AI...


In [13]:
df_phrases["embeddings"] = None

for i, row in df_phrases.iterrows():
  # print(f'Calculating embedding for [{ row["label"] }] {row["example"]}...')
  embedding = calculate_embeddings_with_roberta(row["example"])
  df_phrases.at[i, "embeddings"] = embedding

df_phrases.head(3)

,order,category,label,example,embeddings
0,1,Hello,Hello,"Hello, how are you doing today?","[tensor(0.4046), tensor(0.9384), tensor(0.0146..."
1,1,Hello,Hello,"Hi, everybody.","[tensor(0.1806), tensor(0.8753), tensor(0.2395..."
2,1,Hello,Hello,Hi,"[tensor(0.9531), tensor(0.8997), tensor(-0.060..."


In [14]:
from functools import cmp_to_key
# Split Phrases by Category
df_category_list = [ x for _, x in df_phrases.sort_values(['order','label'],ascending=False).groupby('label') ]
df_category_list.sort(key=cmp_to_key(lambda x, y: 1 if x.iloc[0,0] > y.iloc[0,0] else -1))
print(f"{len(df_category_list)} categories")

6 categories


**Process Transcript**

In [15]:
transcript1 = "Hello, how are you doing today? I'm here to tell you a little bit about, uh, quantified communications and the quantified platform and how it impacts organizations, who it helps and how it works. So I'll get started off by telling you just a little bit about a high level about, um, the quantified platform. Oh, so the quantified platform is one of the most advanced communication intelligence in AI powered coaching systems. And what does that really mean? So, um, communication coaching is something that is typically delivered one on one between a communication coach who has a, uh, a doctorate or a, um, background and experience in teaching people how to be better communicators and how to express themselves effectively. Um, those coaches would work one-on-one with individuals, um, maybe put their information in front of audiences and see how well they respond. And that can be a very costly process as well as a time consuming. And, um, not always backed by the science of what really drives great communication. So the quantified platform allows all of that to be automated through our AI coaching system. Um, our system empowers, um, uh, is empowered by behavioral science in order to be able to take videos into the system and be able to render exactly how a audience is going to perceive you and provide the communication feedback that you need in order to be, become a better communicator. So that helps you sell more, that helps drive better experiences and improves your external communication with your clients. So how does it work? Um, I touched on that a little bit, um, but let me kind of unpack exactly the science behind it. Um, so we started off with a, a large swath of videos of from fantastic communicators, all the people that you would idolize and wanna be like, and we took those videos and we put them in front of panelists and we scored them to see exactly how well they would perform in front of an audience. So how likable, uh, uh, was that speaker, um, how effective were they at communicating their ideas? You know, were they persuasive? Would you actually buy something from them? Did you wanna listen to them longer? Um, did you find them engaging? These things are innately human in their, um, in how communication elicits a response from us? Those are the types of things that we actually measured and built an algorithm around. So the way that the system works is it, um, uh, you are allowed to record yourself inside the application. Um, we also embed into video conferencing platforms as well. So you can invite a bot into your live meeting conversations if you wish. And we have other integration options as well, including having a role playing conversations inside the application. Um, once we have that, the system analyzes, uh, the video content for the words that you say, so your sentence structure, phrases, um, sentiment analysis, pronoun usage, ver burb tone and usage. Um, how you conduct your face, the microexpressions that you have, um, do you appear happy, calm, angry, and your gestures? Um, you know, part of being an appealing, um, conversationalist is being engaging and have people want to, <laugh> want to listen to you. And, and so all of these things all come together into really, um, defining what makes a great piece of communication. And we use that as our benchmark of how to define that inside of our platform. So when you go into the platform, you're really being measured against the best communicators, as well as our entire community of people using the quantified platforms. So you can see where you are against other, um, roles, similar to yours, other people, similar career paths, and see how you grow and, um, get better from there and to optimize your behavior. So who does it help? Um, it can help everybody, everybody can improve their lives, their personal lives, their professional lives, um, their business contacts, their ability to be able to sell and deliver products, um, through having better communications and being able to effectively deliver your message. This is fantastic for people in leadership programs who are looking to accelerate to senior executive executive positions, uh, who are looking to improve their status inside of an organization, their ability to be a leader and be inspiring, um, as well as entry level people who really want to represent their brand well, they wanna have a great impact on their external customer experience, as well as their internal ones. And this, this whole system can be tailored specifically for an organization so that we identify the key characteristics of your top sales leaders, your top performers, your top leaders, and replicate that across the rest of your organization. So it doesn't come with a one size fits all. It is very specific on thinking about the characteristics and the behavioral patterns and the communication styles of those who are already effective inside your organization and creating the patterns to duplicate that. So depending on, on your brand presence and what you value inside of your organization, that can be replicated at scale. So who's it gonna have the greatest impact on, um, those participating in customer experiences, those communicating with customers directly, um, uh, spending time with members of your team, inspiring them, providing leadership guidance, visibility into the overall vision of an organization. Um, there are so much science out there that says really effective leaders lead from great communication. Um, and we wanna remake those people remarkably better. Everybody can improve their communication and everybody deserves to be a great communicator. Um, we see growth early on in the process. So, um, as people participate in the program, they usually, uh, get about 30% better within their first six weeks to 12 weeks. So there's a huge uptick in ability to be able to become more trustworthy, authentic, credible, um, have better collective performances across your team and across your organization and have that individual growth as a team leader. Um, this is all based on evidence based research and a ton of analytics, which we're all very, very proud of. Uh, so I hope that explains our quantified platform. And I look forward to talking to you again soon. Thank you very much."
transcript2 = "Hi, everybody. I'm really excited to be first here to try this out. Not only do I get to go first and be the Guinea pig, but I also know that I will be first on the leaderboard. So tell me about quantified. What is it exactly quantified is an AI powered coaching platform. We are here to make people remarkably better communicators by combining behavioral science, artificial intelligence, and experiential learning to give them the experience of what the world's best communication coach would do with them. If they were sitting with them in every single conversation they had, we know that through our intelligence insights feedback, and we can help people go from not knowing where they stand, when it comes to being how they communicate to becoming exceptional, confident, and inspirational in the way that they communicate, build relationships and succeed professionally in their career. So how does it work? Quantified works by 10, the behaviors that you do, what you do with your voice, your face, your gestures, and your words, and understanding how that an audience is going to react to that we have essentially built a audience digital twin that predicts how people are going to respond to you when you speak. And the technology behind that is really complex, but what's important is the experience is actually pretty simple. We just need a video. So we need a three minute video from you. From those three minutes, we pull out 1400 behaviors and predict audience preference, give you feedback, give you coaching, give you guidance and support your learning journey. So you can go from where you are today to being exceptional in the way that you communicate and speak. Who's helped most by quantified. Well, everybody communicates all day as part of their jobs. We actually study that 80% of your time at work is spent communicating. So who's helped most anyone that talks to customers, anyone that talks to other team members, anyone that talks to people for a living is going to be helped the most, really the more that you communicate as a critical component of your job, the more you're gonna be helped. Finally, how can quantified have the greatest impact on my organization? Well, my organization is quantified, so that's a little bit of a strange answer for me, but, you know, I think directly even what we do for our customers is really important for us, right? Have to represent that we have to represent effective communication to each other and to our clients. And so quantified becoming an exceptional communicator, supporting each other and teaching teaching what great coaching looks like is something that every organization can benefit, certainly including us. And so I'm excited to bring this knowledge and this program to our ourselves and, um, you know, I'm really excited to kick it off. Uh, thank you for allowing me to be first here and let's see how these scores come out. Bye."
transcript3 = "Hi, so upfront caveat, this is going to be terrible. Um, personally, I need days to craft elegant wording, uh, and then more days to practice delivering that message. Um, and instead I'm extemporizing and so this is gonna read much more like a hostage video than marketing material that said dive in. So what is quantified, uh, quantified is the most advanced conversation intelligence and AI powered coaching platform, a software platform that helps people reach their potential, uh, for communicating and connecting empowered by behavioral science, which uses artificial intelligence to drive performance outcomes for customer facing teams, uh, which helps them sell more, which helps them deliver better experiences. How does it work? Um, it is integrated into video conference platforms in which you record yourself, uh, or respond to prompts with fun and realistic experiences. Um, that's all we need you to do, uh, have conversations like the types you have every day. We've built technology that looks at the words that you say, uh, what you do with your voice, your face, your gestures, simple input create, and we create benchmarking, scoring feedback, personalized guidance, and understand how you come across and relay to you how you're doing, uh, how you can get better coach you using in artificial intelligence as if the world's best communication coach. We're sitting there with you in every one of your conversations, uh, telling you how to get better and telling you how to optimize your behavior. So who does it help? Uh, it helps everyone that is having conversations, whether you're an entry level person or the most senior executive, the most powerful group that we can help our customer facing teams, sales teams, customer service teams, customer support, and customer success teams, anyone who is talking to the customer as a core part of their day to day job, uh, words that they use, the way that they present themselves, how they come across has an impact on the performance outcomes of the organization, giving coaching to them, coaching at scale that is standardized and scientific. It's hard to find time for managers to coach, and it is hard for us to give feedback. It's hard for people to feel empowered, to work on something that is personal and private in a safe space. Uh, how can it have the greatest impact, uh, customer experience communicating with the customer, spending time spending a lot of time, communicating with other members of your team, internal communication and external communication. Uh, we wanna make you remarkably better. We want to make you extraordinary at that behavior. Uh, that's meant to be learned along the way. Uh, no one at work can stop you for six months, 12 months, 18 months helping you become exceptional. This is your leadership journey. This is the most important skill for you. They don't have the time and they don't know how to do it. We can provide intentional guidance and deliver that on a personal basis for every single member of your organization that faces the customer. Um, make them 30% better at connecting with other people, measured by evidence based research, uh, the impact that could have on individual performance and your individual growth as a leader, collective performance of the teams in the organization, uh, and make you perceived as trustworthy, authentic, and credible, uh, and improves how you connect."
transcript4 = "My name's Johnny, thank you all so much for taking the time to chat with me today. I kinda wanna just to introduce, quantify, kind of give an overview of who we are and what problems we're trying to seek to solve here. So I'm gonna do this by simply answering four questions. The first of how does it work? Who does it help, who can have the greatest impact? Um, and by answering these questions, I think each of you will have a better understanding of who we are here at quantified. So just to get started, we are the most advanced conversations, intelligence and AI powered coaching platform. We're a software platform that helps people reach their utmost potential and for communicating and connecting with one another, we are empowered by behavioral research and science. So know that everything that we do is backed up by specific science and behavioral research. And we use a component of our artificial intelligence that really drives performance outcomes. And to customer facing teams, we help, for example, sales teams sell more. That's a, an ROI that we're able to capture and we help everyone just deliver overall better experiences to anyone that they're in con um, connection with how does it work? So there's a couple of ways that we can actually give you your objective, personalized feedback. One of which is having a video, um, conference platform integrated into your, um, everyday workflow. So things like zoom or teams, we can actually do real life, um, analysis there. Alternatively, you can record yourself. Um, we always try to give prompts with fun and realistic experiences, and that's all that we'll need to do have conversations like the types you have every day. And we've built that technology that looks at the words that you say, what you do with your voice, your face, your gestures, simple input, and we create benchmarks. That's able to give you that scoring feedback and that personalized guidance for you to be able to improve going forward. So you'll have a good understanding of how you come across a relay back to exactly how you're doing, how you can get better. Um, and coach you using that artificial intelligence. It's able to give you that objective feedback. It's gonna be exactly as if you have the world's best communications coach was sitting, um, there with you in every one of your conversations, telling you how to get better and telling you how to optimize your behavior, who does it help? So literally anyone that has conversations, if you're an entry level person, um, or if you're the most senior executive, you will benefit from our platform, the most powerful group that we can help our customer facing teams, that's sales teams, customer service teams, customer support, and customer success team. Literally anyone who is ha, who is talking to customer has a core part of their day to day job. The words that they use, the way that they present themselves, how they come across, um, that's exactly what we're going to look at and be able to give them coaching, um, based off of their daily interactions. Um, coaching them at a scale is the standardized and scientific. And it's hard to find time for managers to coach, right? So this is something that they're able to do whenever they need to prep for a conversation or, um, in their daily meetings as well. It's hard for people to feel empowered, to work on something that is personal and private. So this gives everyone a safe space to really work on something that's so impactful and so crucial to their day to day job functions. So how can it have the greatest impact customer experience, right? This is going to elevate the way, um, that individuals are communicating with the customer. It's allowing them to spend time on things that matters and impact their overall skillset. We spend 80% of our time communicating. Um, so it's something that we need to work on and progress forward with either other, um, members of your team, internal communication, external communication. We just want to make you remarkably better, right? We want to make your ex, we want to make the conversations that you're having extraordinary meant to be. This is meant to be learning along the way, right? Just like anything else that you do in life practice makes perfect. And this is no exception to that. Um, this is going to help prompt you in your leadership journey. This is the most important skill right now for you. Um, and that's how you're going to be able to perfect and move, um, forward. And through a year of using our platform, we're looking at 30% better at connecting with other people, um, and measured by evidence based research, which is absolutely amazing."

transcript = transcript1

doc = nlp(transcript)
sentences = [ sentence.text for sentence in list(doc.sents) ]
embeddings = model.encode(sentences, convert_to_tensor=True)
print(f"{len(sentences)} sentences")

51 sentences


In [16]:
sentences_mini = [ s[:50] for s in sentences ]

df_results = pd.DataFrame(index=sentences_mini)

for i, df_category in enumerate(df_category_list):
  phrases_list = df_category["embeddings"].values.tolist()
  phrases = torch.stack(phrases_list)
  # Compute cosine-similarities
  cosine_scores = util.cos_sim(embeddings, phrases).numpy()
  max_scores = np.max(cosine_scores, axis=1)
  df_results[df_category.iloc[0,2]] = max_scores

# df_results.set_index('sentence')
df_results

,Hello,What,How,Who,Impact,Bye
"Hello, how are you doing today?",1.000000,0.094632,0.125598,0.271596,0.188807,0.451069
"I'm here to tell you a little bit about, uh, quant",0.278524,0.379432,0.401163,0.193068,0.366785,0.228304
So I'll get started off by telling you just a litt,0.241468,0.350422,0.333086,0.351871,0.212155,0.212886
"Oh, so the quantified platform is one of the most",0.157318,0.742833,0.390680,0.175678,0.252293,0.066303
And what does that really mean?,0.158793,0.037763,0.017822,0.015583,0.035034,0.271494
"So, um, communication coaching is something that i",0.158507,0.408567,0.322659,0.280704,0.452049,0.178120
"Um, those coaches would work one-on-one with indiv",0.111145,0.428433,0.354709,0.273484,0.320183,0.165865
And that can be a very costly process as well as a,0.024777,0.104692,0.071343,0.252281,0.278390,0.094664
"And, um, not always backed by the science of what",0.136516,0.274880,0.170757,0.245018,0.295064,0.161521
So the quantified platform allows all of that to b,0.129321,0.625120,0.319471,0.194983,0.199280,0.098838


In [17]:
import plotly.express as px

fig = px.imshow(
  df_results,
  labels=dict(x="Category", y="Sentence"),
  x=df_results.columns, y=df_results.index,
  text_auto=True,
   aspect="auto",
  color_continuous_scale="rdylgn"
  # color_continuous_scale=px.colors.sequential.YlGn
  # color_continuous_scale=px.colors.sequential.Aggrnyl
  # color_continuous_scale=px.colors.sequential.Viridis
  # color_continuous_scale=px.colors.sequential.BuGn
  # color_continuous_scale=px.colors.sequential.Greens
  # color_continuous_scale=px.colors.diverging.RdYlGn
)

fig.update_layout(height=1000)
fig.update_xaxes(side="top")
fig.show()